## Web Scrapping para obtener datos sobre la poblacion de cada ciudad

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

***Agregamos la url de la pagina y luego comenzamos con extraccion de los datos, especificando donde pueden ser encontrados***

In [2]:
url = 'https://worldpopulationreview.com/cities/united-states'

In [3]:
# Realizamos la solicitud a la página web
response = requests.get(url)

# Verificamos que la solicitud fue exitosa
if response.status_code == 200:
    # Parseamos el contenido HTML de la página
    soup = BeautifulSoup(response.content, 'html.parser')

    # Inicializamos listas para almacenar los datos
    ciudades = []
    poblaciones = []

    # Buscamos todas las filas de la tabla
    filas = soup.find_all('tr', {'data-state': 'false'})

    # Iteramos sobre cada fila y extraemos el nombre de la ciudad y su población
    for fila in filas:
        ciudad = fila.find('th').text.strip()  # El nombre de la ciudad está en <th>
        poblacion = fila.find('td').text.strip()  # La población está en <td>
        
        # Agregamos los datos a las listas
        ciudades.append(ciudad)
        poblaciones.append(poblacion)
    
    # Creamos un DataFrame de pandas con los datos
    df = pd.DataFrame({
        'Ciudad': ciudades,
        'Población': poblaciones
    })
    
    # Mostramos el DataFrame
    print(df)
else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


            Ciudad  Población
0    New York City  8,175,133
1      Los Angeles  3,971,883
2          Chicago  2,720,546
3         Brooklyn  2,300,664
4          Houston  2,296,224
..             ...        ...
395  Boyle Heights     92,785
396     Plantation     92,560
397           Lynn     92,457
398    Miami Beach     92,312
399   Arden-Arcade     92,186

[400 rows x 2 columns]


In [4]:
df.head()

,Ciudad,Población
0,New York City,"8,175,133"
1,Los Angeles,"3,971,883"
2,Chicago,"2,720,546"
3,Brooklyn,"2,300,664"
4,Houston,"2,296,224"


***Filtramos con la lista de ciudades previamente obtenida***

In [5]:
ciudades_a_filtrar = pd.read_csv(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\WEB SCRAPPING\ciudades_california.csv')

# Verificar que la columna de 'Ciudad' existe en el CSV
if 'Ciudad' in ciudades_a_filtrar.columns:
    # Filtrar el DataFrame original, manteniendo solo las ciudades que están en la lista del CSV
    df_filtrado1 = df[df['Ciudad'].isin(ciudades_a_filtrar['Ciudad'])]

    # Mostrar el DataFrame filtrado
    print(df_filtrado1)
else:
    print("El archivo CSV no contiene una columna 'Ciudad'. Verifica el archivo.")

            Ciudad  Población
1      Los Angeles  3,971,883
10       San Diego  1,394,928
13        San Jose  1,026,908
16   San Francisco    864,816
38          Fresno    520,052
..             ...        ...
390       Hesperia     93,295
391         Carson     93,281
393   Santa Monica     93,220
394     San Marcos     92,931
399   Arden-Arcade     92,186

[109 rows x 2 columns]


In [6]:
print(len(df_filtrado1))
df_filtrado1.head

109


<bound method NDFrame.head of             Ciudad  Población
1      Los Angeles  3,971,883
10       San Diego  1,394,928
13        San Jose  1,026,908
16   San Francisco    864,816
38          Fresno    520,052
..             ...        ...
390       Hesperia     93,295
391         Carson     93,281
393   Santa Monica     93,220
394     San Marcos     92,931
399   Arden-Arcade     92,186

[109 rows x 2 columns]>

In [7]:
import re

***Agregamos la url de la pagina de wikipedia y luego comenzamos con extraccion de los datos, limpiandolos en el proceso***

In [8]:
# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Ciudades_de_Estados_Unidos_por_poblaci%C3%B3n"

# Hacemos una solicitud GET para obtener el contenido de la página
response = requests.get(url)

# Analizamos el contenido HTML usando BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Buscamos la tabla correcta (asumimos que es la primera tabla "sortable")
tabla = soup.find('table', {'class': 'sortable'})

# Verificamos que encontramos la tabla
if tabla:
    print("Tabla encontrada correctamente.")
    rows = tabla.find_all('tr')
    print(f"Se encontraron {len(rows) - 1} filas.")  # Excluimos el encabezado
    
    # Listas para almacenar los datos extraídos
    ciudades_wiki = []
    poblaciones_wiki = []

    # Función para limpiar la celda de población desde el final hacia el comienzo
    def limpiar_poblacion(poblacion_texto):
        # Invertimos el texto para trabajar desde el final hacia el comienzo
        poblacion_invertida = poblacion_texto[::-1]
        
        # Tomamos solo los números hasta que encontremos un carácter no deseado (&)
        poblacion_limpia = []
        for char in poblacion_invertida:
            if char.isdigit():
                poblacion_limpia.append(char)
            elif char == '&':
                break  # Detenemos cuando encontramos el primer '&'
        
        # Invertimos el resultado para que vuelva al orden correcto
        poblacion_limpia = ''.join(poblacion_limpia[::-1])
        
        # Convertimos a entero si es posible
        try:
            return int(poblacion_limpia)
        except ValueError:
            return None

    # Iteramos sobre las filas de la tabla (excepto el encabezado)
    for index, row in enumerate(rows[1:], start=1):
        # Extraemos las celdas de cada fila
        cols = row.find_all('td')
        if len(cols) > 0:
            ciudad = cols[2].text.strip()  # La tercera columna contiene la ciudad
            poblacion = cols[4].text.strip()  # La quinta columna contiene la población
            
            # Debug: Ver qué datos estamos extrayendo
            print(f"Fila {index}: Ciudad - {ciudad}, Población (cruda) - {poblacion}")

            # Limpiamos y convertimos la población
            poblacion_limpia = limpiar_poblacion(poblacion)

            # Solo agregamos si la población es válida (no None)
            if poblacion_limpia is not None:
                ciudades_wiki.append(ciudad)
                poblaciones_wiki.append(poblacion_limpia)

    # Convertimos los datos extraídos en un DataFrame
    df_wiki = pd.DataFrame({
        'Ciudad': ciudades_wiki,
        'Población (Wikipedia)': poblaciones_wiki
    })

Tabla encontrada correctamente.
Se encontraron 276 filas.
Fila 1: Ciudad - Nueva York, Población (cruda) - &&&&&&&&08988326.&&&&&08 988 326
Fila 2: Ciudad - Los Ángeles, Población (cruda) - &&&&&&&&03983540.&&&&&03 983 540
Fila 3: Ciudad - Chicago, Población (cruda) - &&&&&&&&02746388.&&&&&02 746 388
Fila 4: Ciudad - Houston, Población (cruda) - &&&&&&&&02472370.&&&&&02 472 370
Fila 5: Ciudad - Phoenix, Población (cruda) - &&&&&&&&01681687.&&&&&01 681 687
Fila 6: Ciudad - San Antonio, Población (cruda) - &&&&&&&&01617461.&&&&&01 617 461
Fila 7: Ciudad - Filadelfia, Población (cruda) - &&&&&&&&01613184.&&&&&01 613 184
Fila 8: Ciudad - San Diego, Población (cruda) - &&&&&&&&01509425.&&&&&01 509 425
Fila 9: Ciudad - Dallas, Población (cruda) - &&&&&&&&01416904.&&&&&01 416 904
Fila 10: Ciudad - San José, Población (cruda) - &&&&&&&&01118577.&&&&&01 118 577
Fila 11: Ciudad - Austin, Población (cruda) - &&&&&&&&01089004.&&&&&01 089 004
Fila 12: Ciudad - Charlotte, Población (cruda) - &&&&&&&

In [9]:
df_wiki

,Ciudad,Población (Wikipedia)
0,Nueva York,8988326
1,Los Ángeles,3983540
2,Chicago,2746388
3,Houston,2472370
4,Phoenix,1681687
...,...,...
271,Erie,101786
272,South Bend,101168
273,Daly City,101123
274,Centennial,100377


In [10]:
print(ciudades_a_filtrar)

         Ciudad
0       Alameda
1        Albany
2      Berkeley
3        Dublin
4    Emeryville
..          ...
770    Woodland
771      Zamora
772  Marysville
773   Wheatland
774       Linda

[775 rows x 1 columns]


***Normalizamos las tablas para poder realizar un merge***

In [11]:
import unidecode

In [12]:
# Función para normalizar los nombres de las ciudades
def normalizar_ciudad(ciudad):
    ciudad = ciudad.strip()  # Eliminar espacios en blanco antes/después
    ciudad = ciudad.lower()  # Convertir a minúsculas
    ciudad = unidecode.unidecode(ciudad)  # Eliminar acentos
    return ciudad

# Normalizamos las ciudades en df_wiki
df_wiki['Ciudad_normalizada'] = df_wiki['Ciudad'].apply(normalizar_ciudad)

# Normalizamos las ciudades en la lista de ciudades_a_filtrar
ciudades_a_filtrar['Ciudad_normalizada'] = ciudades_a_filtrar['Ciudad'].apply(normalizar_ciudad)

# Verificar que la columna 'Ciudad_normalizada' existe en el CSV
if 'Ciudad_normalizada' in ciudades_a_filtrar.columns:
    # Filtrar el DataFrame original, manteniendo solo las ciudades que están en la lista normalizada
    df_filtrado = df_wiki[df_wiki['Ciudad_normalizada'].isin(ciudades_a_filtrar['Ciudad_normalizada'])]

    # Agregar la columna "ciudad" con la primera letra en mayúscula
    df_filtrado['Ciudad'] = df_filtrado['Ciudad_normalizada'].str.title()


C:\Users\Usuario\AppData\Local\Temp\ipykernel_8264\399175488.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Ciudad'] = df_filtrado['Ciudad_normalizada'].str.title()


In [13]:
# Quedarse solo con las columnas "Ciudad" y "Población (Wikipedia)"
df_final = df_filtrado[['Ciudad', 'Población (Wikipedia)']].copy()

# Renombrar la columna "Población (Wikipedia)" a "poblacion"
df_final.rename(columns={'Población (Wikipedia)': 'poblacion'}, inplace=True)

In [14]:
df_final

,Ciudad,poblacion
1,Los Angeles,3983540
7,San Diego,1509425
9,San Jose,1118577
13,San Francisco,928635
32,Fresno,547353
...,...,...
264,Richmond,103701
265,Murrieta,103466
270,Antioch,102372
273,Daly City,101123


In [15]:
df_filtrado1

,Ciudad,Población
1,Los Angeles,"3,971,883"
10,San Diego,"1,394,928"
13,San Jose,"1,026,908"
16,San Francisco,"864,816"
38,Fresno,"520,052"
...,...,...
390,Hesperia,"93,295"
391,Carson,"93,281"
393,Santa Monica,"93,220"
394,San Marcos,"92,931"


In [16]:
# 1. Limpiar la columna de población en df_filtrado1 para eliminar las comas y convertir a enteros
df_filtrado1['Población'] = df_filtrado1['Población'].str.replace(',', '')
df_filtrado1['Población'] = df_filtrado1['Población'].astype(int)

# 2. Convertir la columna de población en df_final a enteros (si no lo es)
df_final['poblacion'] = df_final['poblacion'].astype(int)

# 3. Realizar el merge, agregando las ciudades de df_final que no estén en df_filtrado1
# Usamos outer join para mantener todas las ciudades de ambos DataFrames
df_merged = pd.merge(df_filtrado1, df_final, on='Ciudad', how='outer', indicator=True)

# 4. Si una ciudad está en df_final pero no en df_filtrado1, llenamos los valores faltantes en "Población (df1)"
df_merged['poblacion'].fillna(df_merged['poblacion'], inplace=True)

# 5. Mantener solo las columnas necesarias
df_merged = df_merged[['Ciudad', 'Población']].copy()

# 6. Renombrar la columna 'Población (df1)' a 'poblacion' para uniformidad
df_merged.rename(columns={'Población': 'poblacion'}, inplace=True)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_8264\2133273894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado1['Población'] = df_filtrado1['Población'].str.replace(',', '')
C:\Users\Usuario\AppData\Local\Temp\ipykernel_8264\2133273894.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado1['Población'] = df_filtrado1['Población'].astype(int)


In [17]:
df_merged

,Ciudad,poblacion
0,Albany,98469
1,Alhambra,127764
2,Anaheim,350742
3,Antioch,110542
4,Arden-Arcade,92186
...,...,...
113,Visalia,130104
114,Vista,100890
115,West Covina,108484
116,Westminster,113130


In [20]:
condados_a_ciudades = {
    'Alameda': ['Alameda', 'Albany', 'Berkeley', 'Dublin', 'Emeryville', 'Fremont', 'Hayward', 'Livermore', 'Newark', 'Oakland', 'Pleasanton', 'San Leandro', 'Union City', 'Castro Valley', 'San Lorenzo'],
    'Alpine': ['Markleeville', 'Markleeville'],
    'Amador': ['Amador City', 'Ione', 'Jackson', 'Plymouth', 'Sutter Creek', 'Jackson'],
    'Butte': ['Biggs', 'Chico', 'Gridley', 'Oroville', 'Durham', 'Paradise', 'Chico', 'Magalia', 'Paradise'],
    'Calaveras': ['Angels Camp', 'Copperopolis', 'Arnold', 'San Andreas'],
    'Colusa': ['Colusa', 'Williams', 'Colusa', 'Princeton'],
    'Contra Costa': ['Alameda Park', 'Antioch', 'Brentwood', 'Clayton', 'Concord', 'Crockett', 'Danville', 'El Cerrito', 'Hercules', 'Lafayette', 'Martinez', 'Moraga', 'Orinda', 'Pinole', 'Pittsburg', 'Pleasant Hill', 'Richmond', 'San Pablo', 'San Ramon', 'Walnut Creek', 'Alamo', 'Pacheco', 'Bay Point', 'El Sobrante', 'Oakley', 'Rodeo', 'Discovery Bay'],
    'Del Norte': ['Crescent City'],
    'El Dorado': ['Placerville', 'South Lake Tahoe', 'El Dorado Hills', 'Cool', 'Lotus', 'Cameron Park', 'Kyburz', 'Diamond Springs'],
    'Fresno': ['Biola', 'Clovis', 'Coalinga', 'Firebaugh', 'Fowler', 'Fresno', 'Huron', 'Kerman', 'Kingsburg', 'Mendota', 'Orange Cove', 'Parlier', 'Reedley', 'San Joaquin', 'Sanger', 'Selma', 'Prather', 'Shaver Lake', 'Riverdale', 'Five Points'],
    'Glenn': ['Orland', 'Willows'],
    'Humboldt': ['Arcata', 'Blue Lake', 'Eureka', 'Fortuna', 'Rio Dell', 'Trinidad', 'McKinleyville', 'Ferndale', 'Garberville', 'Loleta', 'Whitethorn', 'Willow Creek'],
    'Imperial': ['Brawley', 'Calexico', 'Calipatria', 'El Centro', 'Holtville', 'Imperial', 'Westmorland'],
    'Inyo': ['Bishop', 'Lone Pine'],
    'Kern': ['Arvin', 'Bakersfield', 'California City', 'Delano', 'Maricopa', 'McFarland', 'Ridgecrest', 'Shafter', 'Taft', 'Tehachapi', 'Wasco', 'Rosamond', 'Edwards AFB', 'Lamont', 'Mojave', 'Lost Hills', 'Kernville', 'Lake Isabella', 'Edwards'],
    'Kings': ['Avenal', 'Corcoran', 'Hanford', 'Lemoore'],
    'Lake': ['Clearlake', 'Lakeport', 'Nice', 'Spring Valley', 'Hidden Valley Lake', 'Kelseyville', 'Lower Lake'],
    'Lassen': ['Susanville', 'Bieber'],
    'Los Angeles': ['Pacific Palisades', 'Valencia', 'Reseda', 'San Pedro', 'City of Industry', 'Northridge', 'North Hollywood', 'Sylmar', 'Sun Valley', 'Artesia', 'Arcadia', 'Venice', 'Sherman Oaks', 'Van Nuys', 'Studio City', 'Granada Hills', 'Rowland Heights', 'Harbor City', 'Tujunga', 'Saugus', 'Highland Park', 'Pacoima', 'Universal City', 'Hollywood', 'East Los Angeles', 'West Hills', 'Cudahy', 'La Crescenta-Montrose', 'Canoga Park','Wilmington', 'Woodland Hills', 'Canyon Country', 'Panorama City', 'Tarzana', 'Winnetka', 'Newhall', 'Castaic', 'Stevenson Ranch', 'Irwindale', 'View Park-Windsor Hills', 'Lomita', 'Marina Del Rey', 'Valley Glen', 'Eagle Rock', 'North Hills', 'Lakewood', 'Arleta', 'Toluca Lake', 'Valley Village', 'Mission Hills', 'Bassett', 'Littlerock', 'Topanga', 'Playa Vista', 'Quartz Hill', 'East Compton', 'Porter Ranch', 'Silver Lake','Agoura Hills', 'Alhambra', 'Alondra Park', 'Altadena', 'Avalon', 'Azusa', 'Baldwin Park', 'Bell', 'Bell Gardens', 'Bellflower', 'Beverly Hills', 'Burbank', 'Calabasas', 'Carson', 'Cerritos', 'Chatsworth', 'Claremont', 'Commerce', 'Compton', 'Covina', 'Culver City', 'Diamond Bar', 'Downey', 'Duarte', 'El Monte', 'El Segundo', 'Encino', 'Gardena', 'Glendale', 'Glendora', 'Hacienda Heights', 'Hawaiian Gardens', 'Hawthorne', 'Hermosa Beach', 'Huntington Park', 'Industry', 'Inglewood', 'La Canada Flintridge', 'La Mirada', 'La Puente', 'La Verne', 'Lancaster', 'Lawndale', 'Lennox', 'Long Beach', 'Los Angeles', 'Lynwood', 'Malibu', 'Manhattan Beach', 'Maywood', 'Monrovia', 'Montebello', 'Monterey Park', 'Norwalk', 'Palmdale', 'Palos Verdes Estates', 'Paramount', 'Pasadena', 'Pico Rivera', 'Pomona', 'Rancho Palos Verdes', 'Redondo Beach', 'Rolling Hills Estates', 'Rosemead', 'San Dimas', 'San Fernando', 'San Gabriel', 'San Marino', 'Santa Clarita', 'Santa Fe Springs', 'Santa Monica', 'Sierra Madre', 'Signal Hill', 'South El Monte', 'South Gate', 'South Pasadena', 'Temple City', 'Torrance', 'Vernon', 'Walnut', 'West Covina', 'West Hollywood', 'Westlake Village', 'Whittier'],
    'Madera': ['Chowchilla', 'Madera', 'Bass Lake', 'North Fork', 'Coarsegold', 'Oakhurst'],
    'Marin': ['Belvedere', 'Corte Madera', 'Fairfax', 'Larkspur', 'Mill Valley', 'Novato', 'Ross', 'San Anselmo', 'San Rafael', 'Sausalito', 'Tiburon', 'Olema', 'Point Reyes Station', 'Greenbrae', 'Stinson Beach', 'Tomales'],
    'Mariposa': ['Mariposa'],
    'Mendocino': ['Fort Bragg', 'Point Arena', 'Ukiah', 'Willits', 'Mendocino', 'Little River', 'Laytonville', 'Philo', 'Gualala', 'Redwood Valley', 'Elk', 'Boonville'],
    'Merced': ['Atwater', 'Dos Palos', 'Gustine', 'Livingston', 'Los Banos', 'Merced', 'Delhi', 'Santa Nella'],
    'Modoc': ['Alturas'],
    'Mono': ['Mammoth Lakes', 'June Lake'],
    'Monterey': ['Carmel-by-the-Sea', 'Del Rey Oaks', 'Gonzales', 'Greenfield', 'King City', 'Marina', 'Monterey', 'Pacific Grove', 'Salinas', 'Sand City', 'Seaside', 'Soledad', 'Carmel Valley', 'Pebble Beach', 'Castroville', 'Pajaro', 'Royal Oaks', 'Prunedale', 'Aromas','Carmel-By-The-Sea'],
    'Napa': ['American Canyon', 'Calistoga', 'Napa', 'St. Helena', 'Yountville', 'St Helena', 'Rutherford', 'Santa Elena'],
    'Nevada': ['Grass Valley', 'Nevada City', 'Truckee', 'Cisco Grove', 'North San Juan', 'Penn Valley'],
    'Orange': ['Aliso Viejo', 'Anaheim', 'Brea', 'Buena Park', 'Costa Mesa', 'Cypress', 'Dana Point', 'Fountain Valley', 'Fullerton', 'Garden Grove', 'Huntington Beach', 'Irvine', 'La Habra', 'La Palma', 'Laguna Beach', 'Laguna Hills', 'Laguna Niguel', 'Laguna Woods', 'Lake Forest', 'Los Alamitos', 'Mission Viejo', 'Newport Beach', 'Orange', 'Placentia', 'Rancho Santa Margarita', 'San Clemente', 'San Juan Capistrano', 'Santa Ana', 'Seal Beach', 'Stanton', 'Tustin', 'Villa Park', 'Westminster', 'Yorba Linda', 'Foothill Ranch', 'Ladera Ranch', 'Silverado', 'Trabuco Canyon', 'Newport Coast'],
    'Placer': ['Auburn', 'Colfax', 'Lincoln', 'Loomis', 'Rocklin', 'Roseville', 'Kings Beach', 'Tahoe City', 'Granite Bay', 'Olympic Valley', 'Newcastle', 'Homewood'],
    'Plumas': ['Portola', 'Quincy', 'Chester', 'Lake Almanor', 'Blairsden-Graeagle'],
    'Riverside': ['Banning', 'Beaumont', 'Blythe', 'Calimesa', 'Canyon Lake', 'Cathedral City', 'Coachella', 'Corona', 'Desert Hot Springs', 'Eastvale', 'Hemet', 'Indian Wells', 'Indio', 'Jurupa Valley', 'La Quinta', 'Lake Elsinore', 'Menifee', 'Moreno Valley', 'Murrieta', 'Norco', 'Palm Desert', 'Palm Springs', 'Perris', 'Rancho Mirage', 'Riverside', 'San Jacinto', 'Temecula', 'Wildomar', 'Thousand Palms', 'Cabazon', 'March Air Reserve Base', 'Idyllwild-Pine Cove', 'Thermal', 'Bermuda Dunes', 'Mira Loma', 'Winchester', 'Anza', 'Mecca', 'Romoland', 'Sun City'],
    'Sacramento': ['Citrus Heights', 'Elk Grove', 'Folsom', 'Galt', 'Isleton', 'Rancho Cordova', 'Sacramento', 'Antelope', 'Arden-Arcade', 'Carmichael', 'Clay', 'Courtland', 'Elverta', 'Fair Oaks', 'Florin', 'Foothill Farms', 'Franklin', 'Fruitridge Pocket', 'Gold River', 'La Riviera', 'Lemon Hill', 'Mather', 'North Highlands', 'Orangevale', 'Parkway', 'Rancho Murieta', 'Rio Linda', 'Rosemont', 'Vineyard', 'Walnut Grove', 'Wilton', 'Hood', 'Rio Linda'],
    'San Benito': ['Hollister', 'San Juan Bautista', 'Tres Pinos'],
    'San Bernardino': ['Adelanto', 'Apple Valley', 'Barstow', 'Big Bear Lake', 'Chino', 'Chino Hills', 'Colton', 'Fontana', 'Grand Terrace', 'Hesperia', 'Highland', 'Loma Linda', 'Montclair', 'Needles', 'Ontario', 'Rancho Cucamonga', 'Redlands', 'Rialto', 'San Bernardino', 'Twentynine Palms', 'Upland', 'Victorville', 'Yucaipa', 'Yucca Valley', 'Joshua Tree', 'Crestline', 'Lake Arrowhead', 'Alta Loma', 'Joshua Tree', 'Oro Grande', 'Yermo', 'Bloomington','Phelan','Helendale','Fort Irwin','Running Springs'],
    'San Diego': ['Carlsbad', 'Chula Vista', 'Coronado', 'Del Mar', 'El Cajon', 'Encinitas', 'Escondido', 'Imperial Beach', 'La Mesa', 'Lemon Grove', 'National City', 'Oceanside', 'Poway', 'San Diego', 'San Marcos', 'Santee', 'Solana Beach', 'Vista', 'Alpine', 'Bonita', 'Borrego Springs', 'Fallbrook', 'Jamul', 'Julian', 'Lakeside', 'Pine Valley', 'Ramona', 'Ranchita', 'Rancho Santa Fe', 'Spring Valley', 'Camp Pendleton North', 'Bonsall', 'Pala', 'Carmel Mountain Ranch', 'Pauma Valley', 'Julian','La Jolla','Camp Pendleton'],
    'San Francisco': ['San Francisco'],
    'San Joaquin': ['Escalon', 'Lathrop', 'Lodi', 'Manteca', 'Ripon', 'Stockton', 'Tracy', 'Acampo', 'French Camp', 'Lockeford', 'Thornton', 'Lockeford', 'Acampo'],
    'San Luis Obispo': ['Arroyo Grande', 'Atascadero', 'Grover Beach', 'Morro Bay', 'Paso Robles', 'Pismo Beach', 'San Luis Obispo', 'Baywood-Los Osos', 'Santa Margarita', 'Cayucos', 'Oceano', 'Templeton', 'Nipomo', 'Cambria', 'San Simeon', 'Los Osos'],
    'San Mateo': ['Atherton', 'Belmont', 'Brisbane', 'Burlingame', 'Colma', 'Daly City', 'East Palo Alto', 'Foster City', 'Half Moon Bay', 'Hillsborough', 'Menlo Park', 'Millbrae', 'Pacifica', 'Portola Valley', 'Redwood City', 'San Bruno', 'San Carlos', 'San Mateo', 'South San Francisco', 'Woodside'],
    'Santa Barbara': ['Buellton', 'Carpinteria', 'Goleta', 'Guadalupe', 'Lompoc', 'Santa Barbara', 'Santa Maria', 'Solvang', 'Orcutt', 'Los Alamos','Los Olivos','Montecito','Isla Vista','Vandenberg AFB'],
    'Santa Clara': ['Campbell', 'Cupertino', 'Gilroy', 'Los Altos', 'Los Altos Hills', 'Los Gatos', 'Milpitas', 'Monte Sereno', 'Morgan Hill', 'Mountain View', 'Palo Alto', 'San Jose', 'Santa Clara', 'Saratoga', 'Sunnyvale','Stanford'],
    'Santa Cruz': ['Capitola', 'Santa Cruz', 'Scotts Valley', 'Watsonville', 'Boulder Creek', 'Freedom','Aptos','Soquel','Felton'],
    'Shasta': ['Anderson', 'Redding', 'Shasta Lake', 'Burney', 'McArthur', 'Platina', 'Lakehead', 'Fall River Mills', 'Cottonwood'],
    'Sierra': ['Loyalton', 'Sierra City'],
    'Siskiyou': ['Dorris', 'Dunsmuir', 'Etna', 'Fort Jones', 'Montague', 'Mount Shasta', 'Tulelake', 'Weed', 'Yreka', 'Mt Shasta', 'Mount Shasta'],
    'Solano': ['Benicia', 'Dixon', 'Fairfield', 'Rio Vista', 'Suisun City', 'Vacaville', 'Vallejo'],
    'Sonoma': ['Cloverdale', 'Cotati', 'Healdsburg', 'Petaluma', 'Rohnert Park', 'Santa Rosa', 'Sebastopol', 'Sonoma', 'Windsor', 'Bodega Bay', 'Guerneville', 'Geyserville', 'Forestville', 'Monte Rio'],
    'Stanislaus': ['Ceres', 'Hughson', 'Modesto', 'Newman', 'Oakdale', 'Patterson', 'Riverbank', 'Turlock', 'Waterford', 'Salida', 'Keyes', 'Westley'],
    'Sutter': ['Live Oak', 'Yuba City', 'Sutter'],
    'Tehama': ['Corning', 'Red Bluff', 'Tehama'],
    'Trinity': ['Weaverville', 'Wildwood', 'Lewiston'],
    'Tulare': ['Dinuba', 'Exeter', 'Farmersville', 'Lindsay', 'Porterville', 'Tulare', 'Visalia', 'Woodlake', 'Alpaugh', 'Orosi', 'Springville','Earlimart','Pixley','Terra Bella','Ivanhoe','Strathmore','Richgrove'],
    'Tuolumne': ['Sonora', 'Jamestown', 'Columbia', 'Pinecrest', 'Groveland'],
    'Ventura': ['Camarillo', 'Fillmore', 'Moorpark', 'Ojai', 'Oxnard', 'Port Hueneme', 'Santa Paula', 'Simi Valley', 'Thousand Oaks', 'Ventura', 'Oak Park','Newbury Park','Somis'],
    'Yolo': ['Davis', 'West Sacramento', 'Winters', 'Woodland', 'Zamora'],
    'Yuba': ['Marysville', 'Wheatland','Linda']
}

In [22]:
# Crear una nueva columna 'County' con valores vacíos
df_merged['County'] = None

# Recorrer el diccionario y asignar el condado correspondiente
for condado, ciudades in condados_a_ciudades.items():
    df_merged.loc[df_merged['Ciudad'].isin(ciudades), 'County'] = condado

# Verificar el resultado
print(df_merged.head())

         Ciudad  poblacion        County
0        Albany      98469       Alameda
1      Alhambra     127764   Los Angeles
2       Anaheim     350742        Orange
3       Antioch     110542  Contra Costa
4  Arden-Arcade      92186    Sacramento


In [23]:
# Guardar el DataFrame en un archivo Parquet en una ruta específica
ruta = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\poblacion_ciudades_california.csv'
df_merged.to_csv(ruta, index=False)
